In [1]:
import pandas as pd
import numpy as np
import os
import nibabel as nib
from tqdm.notebook import tqdm

In [2]:
base_dir = "../data/MMs/"

In [3]:
def load_nii(img_path):
    """
    Function to load a 'nii' or 'nii.gz' file, The function returns
    everyting needed to save another 'nii' or 'nii.gz'
    in the same dimensional space, i.e. the affine matrix and the header
    :param img_path: (string) Path of the 'nii' or 'nii.gz' image file name
    :return: Three element, the first is a numpy array of the image values (height, width, slices, phases),
             ## (No) the second is the affine transformation of the image, and the
             ## (No) last one is the header of the image.
    """
    nimg = nib.load(img_path)
    return np.asanyarray(nimg.dataobj), nimg.affine, nimg.header

In [4]:
volumes_df = pd.read_csv("../data/MMs/volume_info.csv")
volumes_df.head()

,External code,VendorName,Vendor,Centre,ED,ES,Partition,Labeled
0,A0S9V9,Siemens,A,1,0,9,Training,True
1,A1D0Q7,Philips,B,2,0,9,Training,True
2,A1D9Z7,Siemens,A,1,22,11,Training,True
3,A1E9Q1,Siemens,A,1,0,9,Training,True
4,A1K2P5,Canon,D,5,33,11,Testing,False


In [5]:
volumes_df["Vol_mean"] = -1.0
volumes_df["Vol_std"] = -1.0
volumes_df["ES_mean"] = -1.0
volumes_df["ES_std"] = -1.0
volumes_df["ED_mean"] = -1.0
volumes_df["ED_std"] = -1.0

volumes_df["Vol_max"] = -1.0
volumes_df["Vol_min"] = -1.0
volumes_df["ES_max"] = -1.0
volumes_df["ES_min"] = -1.0
volumes_df["ED_max"] = -1.0
volumes_df["ED_min"] = -1.0

In [6]:
for i, row in tqdm(volumes_df.iterrows(), total=volumes_df.shape[0]):

    external_code = row["External code"]
    partition = row["Partition"]
    
    if partition == "Training":
        label_str = "Labeled" if row["Labeled"] else "Unlabeled"
        partition = f"{partition}/{label_str}"
    
    nifti_path = os.path.join(
        base_dir, partition, external_code, f"{external_code}_sa.nii.gz"
    )
    nifti_volume = load_nii(nifti_path)[0]
    h,w, c_slices, c_phases = nifti_volume.shape  # h, w, slices, *phases*
    
    volumes_df.at[i,'Vol_mean'] = np.mean(nifti_volume)
    volumes_df.at[i,'Vol_std'] = np.std(nifti_volume)
    volumes_df.at[i,'Vol_max'] = np.max(nifti_volume)
    volumes_df.at[i,'Vol_min'] = np.min(nifti_volume)
    
    volumes_df.at[i,'ES_mean'] = np.mean(nifti_volume[..., row["ES"]])
    volumes_df.at[i,'ES_std'] = np.std(nifti_volume[..., row["ES"]])
    volumes_df.at[i,'ES_max'] = np.max(nifti_volume[..., row["ES"]])
    volumes_df.at[i,'ES_min'] = np.min(nifti_volume[..., row["ES"]])
    
    volumes_df.at[i,'ED_mean'] = np.mean(nifti_volume[..., row["ED"]])
    volumes_df.at[i,'ED_std'] = np.std(nifti_volume[..., row["ED"]])
    volumes_df.at[i,'ED_max'] = np.max(nifti_volume[..., row["ED"]])
    volumes_df.at[i,'ED_min'] = np.min(nifti_volume[..., row["ED"]])

In [7]:
volumes_df.head()

,External code,VendorName,Vendor,Centre,ED,ES,Partition,Labeled,Vol_mean,Vol_std,ES_mean,ES_std,ED_mean,ED_std,Vol_max,Vol_min,ES_max,ES_min,ED_max,ED_min
0,A0S9V9,Siemens,A,1,0,9,Training,True,172.248077,214.853607,172.566757,214.829666,172.608582,215.067871,2629.0,0.0,2621.0,0.0,2593.0,0.0
1,A1D0Q7,Philips,B,2,0,9,Training,True,82.365715,107.156891,82.095421,106.759872,83.263290,108.002403,1283.0,0.0,955.0,0.0,684.0,0.0
2,A1D9Z7,Siemens,A,1,22,11,Training,True,135.064224,109.693542,134.814407,109.553276,135.399307,109.811104,2286.0,0.0,2271.0,0.0,2260.0,0.0
3,A1E9Q1,Siemens,A,1,0,9,Training,True,38.451416,36.569950,38.294273,36.254002,37.931664,36.011513,272.0,0.0,234.0,0.0,206.0,0.0
4,A1K2P5,Canon,D,5,33,11,Testing,False,1059.748535,1732.398804,1062.826904,1739.140747,1057.126343,1729.502441,16383.0,0.0,16383.0,0.0,12914.0,0.0


In [8]:
volumes_df.to_csv("../data/MMs/volume_info_statistics.csv", index=False)